- Create a distribution of datapoints that are separable based on categories
- Show the distribution of points with and without categories
- Run the stats, show what the F-statistic looks like, and how we get it
- Effect size and how we get it

In [ ]:
# Imports for analyzing and visualizing data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns